In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('spotify_millsongdata.csv')

In [4]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.shape

(5000, 3)

In [10]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [11]:
df['text']

0       love, love me do.  \r you know i love you,  \r...
1       isn't it like me  \r to want to say  \r things...
2       he is good  \r they are happy  \r he is strong...
3       hark the herald angels sing  \r glory to the n...
4       you're far away from me  \r and we don't talk ...
                              ...                        
4995    words by marilyn and alan bergman, music by mi...
4996    remember the nights we sat  \r and talked abou...
4997    i found him by the railroad tracks this mornin...
4998    burn all the candles out  \r make a wish but n...
4999    it's fine  \r fine, fine, fine, fine  \r it's ...
Name: text, Length: 5000, dtype: object

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [13]:
stemmer = PorterStemmer()

In [14]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [15]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [16]:
df['text'].apply(lambda x: token(x))

0       love , love me do . you know i love you , i 'l...
1       is n't it like me to want to say thing i ca n'...
2       he is good they are happi he is strong they ar...
3       hark the herald angel sing glori to the new bo...
4       you 're far away from me and we do n't talk en...
                              ...                        
4995    word by marilyn and alan bergman , music by mi...
4996    rememb the night we sat and talk about all our...
4997    i found him by the railroad track thi morn i c...
4998    burn all the candl out make a wish but not alo...
4999    it 's fine fine , fine , fine , fine it 's alr...
Name: text, Length: 5000, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.05484185, 0.02596431, ..., 0.09681966, 0.00401693,
       0.04203385])

In [20]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))